# Tuning the reaction wheel PID controller
#### V. Hunter Adams

This webpage provides the recommended procedure for tuning the PID controller which balances the [reaction wheel inverted pendulum](./ReactionWheel.html). For more general information about PID controllers, see:

- [Phenomenological introduction to PID controllers](https://vanhunteradams.com/PID/PID.html)
- [Analytical introduction to PID controllers](https://vanhunteradams.com/PID/PID_Analytical.html)

***

## Procedure

### 1. Get the complementary filter running!
You cannot build a controller with a measurement of the arm's tilt angle. Start by implementing a [complementary filter](./Complementary_Filters.html) of the accelerometer and gyroscope measurements running to estimate angle.

### 2. Implement proportional control
The proportional term of the PID controller sets the voltage (or PWM duty cycle) applied to the motor by scaling the error between the estimated tilt angle (from the complementary filter) and the desired tilt angle (0 degrees) by a constant $K_p$.

If you are starting from the [PWM demo](https://github.com/vha3/Hunter-Adams-RP2040-Demos/tree/master/Lab_3/PWM_Demo) in the course demo code repository, then the PWM channel can be assigned a duty cycle in the range [0, 5000]. We want for the controller to work over a range of +/- 3-4 degrees off the vertical, so start with a proportional gain of approximately 1500. Here is some pseudocode:

```c
// Compute the error
error = (desired_angle - complementary_angle) ;

// Compute duty cycle with P controller
duty_cycle = Kp * error ; 
```

Use your hand to gently move the pendulum back and forth. Do you see the reaction wheel speed increase in one direction when you move the arm right, and in the other direction when you move it left? Do you feel the arm fighting against your rotation, or trying to make the arm rotate faster? If it feels like the wheel is trying to pull the arm faster, then switch the sign on $K_p$.

Your calculation of the duty cycle may yield a negative number. Remember that the sign of the duty cycle just indicates the *direction* that the motor should turn. In other words, the sign tells you which of the two PWM channels which drive the H-bridge you should command, and which you should set to zero.

### 3. Add integral control

```c
// Compute the error
error = (desired_angle - complementary_angle) ;

// Integrate the error
error_accumulation += error ;

// Clamp the integrated error (start with Imax = max_duty_cycle/2)
if (error_accumulation>Imax) error_accumulation=Imax ;
if (error_accumulation<(-Imax)) error_accumulation=-Imax ;

// Compute duty cycle with PI controller
duty_cycle = (Kp * error) + (Ki * error_accumulation) ; 
```

### 4. Add derivative control

```c
// Compute the error
error = (desired_angle - complementary_angle) ;

// Integrate the error
error_accumulation += error ;

// Clamp the integrated error (start with Imax = max_duty_cycle/2)
if (error_accumulation>Imax) error_accumulation=Imax ;
if (error_accumulation<(-Imax)) error_accumulation=-Imax ;

// Approximate the rate of change of the error
error_deriv = (error - prev_error) ;

// Compute duty cycle with PID controller
duty_cycle = (Kp * error) + (Ki * error_accumulation) + (Kd * error_deriv) ; 

// Update prev_error
prev_error = error ;
```

### 5. Add dithering

```c
// Compute the error
error = (desired_angle - complementary_angle) ;

// Start with angle_increment = 0.0001
if (error < 0) {
    desired_angle -= angle_increment ;
}
else {
    desired_angle += angle_increment ;
}

// Integrate the error
error_accumulation += error ;

// Clamp the integrated error (start with Imax = max_duty_cycle/2)
if (error_accumulation>Imax) error_accumulation=Imax ;
if (error_accumulation<(-Imax)) error_accumulation=-Imax ;

// Approximate the rate of change of the error
error_deriv = (error - prev_error) ;

// Compute duty cycle with PID controller
duty_cycle = (Kp * error) + (Ki * error_accumulation) + (Kd * error_deriv) ; 

// Update prev_error
prev_error = error ;
```